In [18]:
import pandas
import itertools as it

DS_FOR_KIDS = True


if DS_FOR_KIDS:
    lookup_table = pandas.read_csv("./legacy/valid_values_kids.csv", sep=";")
else:
    lookup_table = pandas.read_csv("./legacy/valid_values.csv", sep = ";")

print(lookup_table[0:3])

   NumChickens FieldArea ItemSurfaceArea  Unnamed: 3       Radius  \
0           21  17777,78           49,25         NaN   0,80000005   
1           25  20069,44           49,25         NaN  0,849999906   
2           37     22500           49,25         NaN          0,9   

          Size   ASB  Unnamed: 7  Unnamed: 8  Unnamed: 9  ...  Unnamed: 16  \
0  4,000165622  0,85         NaN         NaN         NaN  ...          NaN   
1  4,000165622  0,85         NaN         NaN         NaN  ...          NaN   
2  4,000165622  0,85         NaN         NaN         NaN  ...          NaN   

   Unnamed: 17  Unnamed: 18  Unnamed: 19  Unnamed: 20  Unnamed: 21  \
0          NaN          NaN          NaN          NaN          NaN   
1          NaN          NaN          NaN          NaN          NaN   
2          NaN          NaN          NaN          NaN          NaN   

   Unnamed: 22  Unnamed: 23  Unnamed: 24  Unnamed: 25  
0          NaN          NaN          NaN          NaN  
1          NaN   

In [ ]:
lookup_table.reset_index()
for i, data in lookup_table.iterrows():
    if i > 0:
        break
    print(i)
    #print(data)
    print(data[1].replace(",", "."))

In [14]:
from sklearn.decomposition import TruncatedSVD
from sklearn.decomposition import PCA
import numpy as np
from typing import Tuple, List

def get_PCA_pars(valid_values: pandas.DataFrame) -> Tuple[TruncatedSVD, np.ndarray]:
    pars = []
    j=0
    for i,t in valid_values.iterrows():
        pars.append([float(t[1].replace(",", ".")), float(t[2].replace(",", "."))])
        j=i
    pars = np.array(pars)

    #print(i)
    mean = pars.mean(axis=0)
    #print(mean)
    pars = pars/mean
    pca = TruncatedSVD(n_components=1)

    pca.fit(pars)

    return pca, mean

def compute_ND_NND(pca: TruncatedSVD, mean:np.ndarray, t_right: List[float], t_left: List[float]) -> Tuple[float, float]:
    n_right = t_right[0]
    n_left = t_left[0]

    t_right_np = np.array([[t_right[1], t_right[2]]])/mean
    t_left_np = np.array([[t_left[1], t_left[2]]])/mean

    #print(t_right_np)

    sd_right = pca.transform(t_right_np)[0][0]
    sd_left = pca.transform(t_left_np)[0][0]


    return np.log(n_right/n_left), np.log(sd_right/sd_left) 

In [15]:
import random
data = {}
columns = ["Unnamed","NumLeft","FieldAreaLeft","ItemSurfaceAreaLeft","NumRight","FieldAreaRight","ItemSurfaceAreaRight","Ratio L/R","LogRatio","Log - Normalized","Difficulty Coefficient","Diff_coeff_filtering", "nd_LogRatio", "nnd_LogRatio"]
pca, mean = get_PCA_pars(lookup_table)
for c in columns:
    data[c] = []
i=0
for (ileft, dleft), (iright,dright) in it.combinations(lookup_table.iterrows(), 2):
    if dleft[0] == dright[0]:
        continue
    #print(dleft)
    fAreaLeft = float(dleft[1].replace(",", "."))
    fAreaRight = float(dright[1].replace(",", "."))
    iAreaLeft = float(dleft[2].replace(",", "."))
    iAreaRight = float(dright[2].replace(",", "."))

    data["Unnamed"].append(i) 
    data["NumLeft"].append(dleft[0])
    data["FieldAreaLeft"].append(fAreaLeft)
    data["ItemSurfaceAreaLeft"].append(iAreaLeft)
    data["NumRight"].append(dright[0])
    data["FieldAreaRight"].append(fAreaRight)
    data["ItemSurfaceAreaRight"].append(iAreaRight)

    #nd and nnd coords, should be relevant for any kind of implemented ai 
    nd_logratio, nnd_logratio = compute_ND_NND(pca, mean,[int(dleft[0]), fAreaLeft,iAreaLeft], [int(dright[0]), fAreaRight, iAreaRight])
    data["nd_LogRatio"].append(nd_logratio)
    data["nnd_LogRatio"].append(nnd_logratio)

    #old statistics, computed for backward compatibility
    data["Ratio L/R"].append(round((dleft[0]/dright[0]),2))
    data["LogRatio"].append(round(np.log10(round((dleft[0]/dright[0]),2)),2))
    data["Log - Normalized"].append(round(np.log10(round((dleft[0]/dright[0]),2)),2))
    data["Difficulty Coefficient"].append(round(random.uniform(0, 1), 2))
    data["Diff_coeff_filtering"].append(round(random.uniform(0,1), 2))


    #do the same for the specular trial
    data["Unnamed"].append(i) 
    data["NumLeft"].append(dright[0])
    data["FieldAreaLeft"].append(fAreaRight)
    data["ItemSurfaceAreaLeft"].append(iAreaRight)
    data["NumRight"].append(dleft[0])
    data["FieldAreaRight"].append(fAreaLeft)
    data["ItemSurfaceAreaRight"].append(iAreaLeft)

    #nd and nnd coords, should be relevant for any kind of implemented ai 
    nd_logratio, nnd_logratio = compute_ND_NND(pca,mean,[int(dright[0]), fAreaRight,iAreaRight], [int(dleft[0]), fAreaLeft, iAreaLeft])
    data["nd_LogRatio"].append(nd_logratio)
    data["nnd_LogRatio"].append(nnd_logratio)

    #old statistics, computed for backward compatibility
    data["Ratio L/R"].append(round((dright[0]/dleft[0]),2))
    data["LogRatio"].append(round(np.log10(round((dright[0]/dleft[0]),2)),2))
    data["Log - Normalized"].append(round(np.log10(round((dright[0]/dleft[0]),2)),2))
    data["Difficulty Coefficient"].append(round(random.uniform(0, 1), 2))
    data["Diff_coeff_filtering"].append(round(random.uniform(0,1), 2))

In [16]:
import math
new_table = pandas.DataFrame(data = data, columns=columns)

# additional processing for backward compatibility
def diff_coef(i, x, y):
    try:
        if (x > 0 and y >= 0) or (x < 0 and y <= 0):
            k = math.atan(y/x)
        elif (x < 0 and y >= 0) or (x > 0 and y <= 0):
            k = math.pi - abs(math.atan(y/x))
        a = math.degrees(abs((math.pi/4)-k))
    except:
        return -1
    return a

#i think it's wrong normalizing and making decisions on normalized values,
#adding new trials can change the whole feature space, we are losing coherency with respect to Learning to Focus on Numbers
new_table["Log - Normalized"] = round(new_table["Log - Normalized"]/new_table["Log - Normalized"].max(),2).abs()
new_table["Difficulty Coefficient"] = round(1- new_table["Log - Normalized"],2)

max_nd = new_table['nd_LogRatio'].abs().max()
max_nnd = new_table['nnd_LogRatio'].abs().max()

filtering_diff = []
for i, row in new_table.iterrows():
    filtering_diff.append(diff_coef(i,row["nd_LogRatio"]/max_nd, row["nnd_LogRatio"]/max_nnd))

new_table["Diff_coeff_filtering"] = filtering_diff
new_table["Diff_coeff_filtering"] = new_table["Diff_coeff_filtering"]/new_table["Diff_coeff_filtering"].max()

In [17]:
import os

if DS_FOR_KIDS:
    if os.path.exists("lookup_table_kids.csv"):
        os.remove("lookup_table_kids.csv")
    new_table.to_csv("./lookup_table_kids.csv")
else:
    if os.path.exists("lookup_table.csv"):
        os.remove("lookup_table.csv")
    new_table.to_csv("./lookup_table.csv")

# Some analysis and ad hoc computations

In [ ]:
import pandas
lookup_table = pandas.read_csv("./lookup_table.csv")

print(lookup_table['nd_LogRatio'].min())
print(lookup_table['nd_LogRatio'].max())
print(lookup_table['nnd_LogRatio'].min())
print(lookup_table['nnd_LogRatio'].max())
#highest = lookup_table.iloc((lookup_table['nd_LogRatio']).abs().argsort()[:1])

In [ ]:
okay = False
count = 0
count_two=0
for row in lookup_table.iterrows():
    row=row[1]
    if row["nd_LogRatio"] >0 and row["nnd_LogRatio"]<0:
        #print(row)
        okay = True
        #assert False == True
        count+=1
    if row["nd_LogRatio"] <0 and row["nnd_LogRatio"]>0:
        count_two +=1

print(okay)
print(count)
print(count_two)

In [ ]:
max_nd = lookup_table['nd_LogRatio'].abs().max()
max_nnd = lookup_table['nnd_LogRatio'].abs().max()

lookup_table['nd_LogRatio'] = lookup_table['nd_LogRatio'].abs()/max_nd
lookup_table['nnd_LogRatio'] = lookup_table['nnd_LogRatio'].abs()/max_nnd

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
def myHistogram(data: np.ndarray,n_labels:int,labels:np.ndarray, bins: int=20):
    colors = ["red", "green", "blue","orange" , "gray", "orange"]

    for i in range(0,data.shape[0]):
        for j in range(0,n_labels):
            plt.hist(data[i, labels == j],bins=bins, density=True, color=colors[j],alpha=0.6, histtype='bar', rwidth=0.9)
        plt.show()

In [ ]:
nd_logratio = lookup_table["nd_LogRatio"]
nnd_logratio = lookup_table["nnd_LogRatio"]

nd =plt.hist(nd_logratio, color="red", histtype='bar', alpha=0.6)
nnd =plt.hist(nnd_logratio, color = "green",  histtype='bar', alpha=0.6)

plt.legend(["Numerical Dimension", ["Non Numerical Dimension"]])

plt.show()

In [ ]:
import pandas
import math
def diff_coeff(i, x, y):
    
    if (x > 0 and y >= 0) or (x < 0 and y <= 0):
            k = math.atan(y/x)
    elif (x < 0 and y >= 0) or (x > 0 and y <= 0):
        k = math.pi - abs(math.atan(y/x))
    a = math.degrees(abs((math.pi/4)-k))
    
    return a

new_table = pandas.read_csv("./lookup_table.csv")

max_nd = new_table['nd_LogRatio'].abs().max()
max_nnd = new_table['nnd_LogRatio'].abs().max()
print(diff_coeff(1, 0.3, 0.2))
print(diff_coeff(1, -0.3, 0.2))

In [ ]:
import numpy as np

import matplotlib.pyplot as plt
from typing import List, Any, Callable

def to_mock_trial(nd: float, nnd: float):
    return [-1,-1,-1,-1,-1,-1,-1,-1,nd, nnd]

def plot_trials(boundary_vector: np.ndarray, trials: List[List[Any]], corrects: List[bool], annotations: List[float], ann_str: bool = False, plot_stats: Callable = None, plot_dist: bool = False):

    fig, ax= plt.subplots(1,1)
    #ax = fig.axes
    
    vec= 5*boundary_vector
    
    plt.plot([vec[0], -vec[0]], [vec[1], -vec[1]])

    colors = {True: 'green', False: 'red'}

    coords = list(map(lambda x: x[8:],trials))
    
    
    for i, coord in enumerate(coords):
        #print(corrects[i])
        plt.scatter(coord[0], coord[1], color = colors[corrects[i]])
        #ax.text(coord[0]-0.1, coord[1]+0.1, str(round(times[i],2)), color = colors[corrects[i]])

        if ann_str:
            plt.annotate(annotations[i], (coord[0], coord[1]), color= "red")
        else:    
            ax.annotate(str(round(annotations[i],2)), (coord[0], coord[1]), color="red")
        #print(f">>{coord}")
        if plot_dist:
            coord = np.array(coord)
            proj = np.dot(boundary_vector, coord)
            proj = boundary_vector*proj
            ax.plot([proj[0], coord[0]], [proj[1], coord[1]], color="red")
        
    if plot_stats is not None:
        plot_stats(plt)

    ax.spines['left'].set_position('zero')
    ax.spines['right'].set_color('none')
    ax.spines['bottom'].set_position('zero')
    ax.spines['top'].set_color('none')

    plt.xlim([-1, 1])
    plt.ylim([-1,1])
    #fig.supxlabel("Numerical dimension", loc="left")
    #fig.supylabel("Non numerical dimension", loc="bottom")
    plt.grid(True)

    plt.show()

def unit_vector(vector):
    """ Returns the unit vector of the vector.  """
    return vector / np.linalg.norm(vector)

In [ ]:
import pandas
import math
#from ..AI.ai_plot import plot_trials

lookup_table = pandas.read_csv("./lookup_table.csv")

max_nd = lookup_table['nd_LogRatio'].abs().max()
max_nnd = lookup_table['nnd_LogRatio'].abs().max()

col = "Diff_coeff_filtering"
fetched_samples = 1

target_main_diff = 0.95
hard = lookup_table.iloc[(lookup_table[col]-target_main_diff).abs().argsort()[:fetched_samples]]

target_main_diff = 0.01
easy = lookup_table.iloc[(lookup_table[col]-target_main_diff).abs().argsort()[:fetched_samples]]

hard_diff = round(float(hard[col]),2)
easy_diff = round(float(easy[col]),2) 

hard = to_mock_trial(float(hard['nd_LogRatio'])/max_nd, float(hard['nnd_LogRatio'])/max_nnd)
easy = to_mock_trial(float(easy['nd_LogRatio'])/max_nd, float(easy['nnd_LogRatio'])/max_nnd)


print(hard)
print(easy)

In [ ]:
trials = [hard, easy]

corrects = [True, True]
anns = [f"fd - {hard_diff}", f"fd - {easy_diff}"]

plot_trials([1, 1], trials, corrects, anns, ann_str=True)

In [ ]:
import numpy as np
col = "Difficulty Coefficient"
fetched_samples = 1

x = np.linspace(0.2, 0.8, 5)

trials = []
corrects = []
anns = []

for sd in x:
    target_main_diff = sd
    trial = lookup_table.iloc[(lookup_table[col]-target_main_diff).abs().argsort()[:fetched_samples]]
    diff = round(float(trial[col]),2)

    trials.append(to_mock_trial(float(trial['nd_LogRatio'])/max_nd, float(trial['nnd_LogRatio'])/max_nnd))
    corrects.append(True)
    anns.append(f"sd - {round(float(trial[col]),2) }")

print(easy)